In [1]:
import xarray as xr
from collections import OrderedDict


url = "http://barataria.tamu.edu:8080/thredds/dodsC/NcML/txla_hindcast_agg"
ds = xr.open_dataset(url)
# Turn on chunking to activate dask and parallelize read/write
ds = ds.chunk({})

In [2]:
def filter_by(self, **kwargs):
    selection = []
    for var_name, variable in self.variables.items():
        has_value_flag = False
        for attr_name, pattern in kwargs.items():
            attr_value = variable.attrs.get(attr_name)
            if ((callable(pattern) and pattern(attr_value))
                    or attr_value == pattern):
                has_value_flag = True
            else:
                has_value_flag = False
                break
        if has_value_flag is True:
            selection.append(var_name)
    return self[selection]

In [3]:
def get_formula_terms(var):
    formula_terms = OrderedDict()
    var = var.attrs.get("formula_terms")
    terms = [x.strip(":") for x in var.split()]
    for k, v in zip(terms[::2], terms[1::2]):
        formula_terms.update({k: v})
    return formula_terms

In [4]:
def ocean_s_coordinate_g2(s, eta, depth, depth_c, c):
    """
    Creates a dimensioned version of s-coordinate generic form 2.
    z(n,k,j,i) = eta(n,j,i) + (eta(n,j,i) + depth(j,i)) * S(k,j,i)
    where:
        S(k,j,i) = (depth_c * s(k) + depth(j,i) * C(k)) /
                    (depth_c + depth(j,i))
    """
    S = (depth_c * s + depth * c) / (depth_c + depth)
    return eta + (eta + depth) * S

In [5]:
S = filter_by(ds, standard_name="ocean_s_coordinate_g2")  # need to fix filter_by to find the coords
S = S["s_rho"]  # need to find this automagically

map_s = get_formula_terms(S)

In [6]:
my_map_s = {k.lower(): ds[map_s[k]] for k, v in map_s.items()}

In [7]:
z = ocean_s_coordinate_g2(**my_map_s)
z

<xarray.DataArray (ocean_time: 223487, eta_rho: 191, xi_rho: 671, s_rho: 30)>
dask.array<shape=(223487, 191, 671, 30), dtype=float64, chunksize=(223487, 191, 671, 30)>
Coordinates:
    lon_rho     (eta_rho, xi_rho) float64 dask.array<shape=(191, 671), chunksize=(191, 671)>
    lat_rho     (eta_rho, xi_rho) float64 dask.array<shape=(191, 671), chunksize=(191, 671)>
  * ocean_time  (ocean_time) datetime64[ns] 1993-01-01T01:00:00 ... 2018-07-01
  * s_rho       (s_rho) float64 -0.9833 -0.95 -0.9167 ... -0.05 -0.01667
Dimensions without coordinates: eta_rho, xi_rho